In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
sample_sub = pd.read_csv("sample_submission.csv")
train_df = pd.read_csv("train (1).csv")
test_df = pd.read_csv("test (1).csv")

In [5]:
X_train = train_df.drop(columns={'Id','Property_A','Property_B','Property_C','Property_D','Property_E','Property_F'})
y_train = train_df[['Property_A','Property_B','Property_C','Property_D','Property_E','Property_F']]

In [6]:
sensor1_X_train = X_train[['A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']]

In [7]:
sensor2_X_train = X_train.drop(columns={'A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'})

In [101]:
copy_sensor2_X_train = sensor2_X_train


# https://lifewithdata.com/2022/03/13/how-to-remove-highly-correlated-features-from-a-dataset/
corr_matrix = sensor2_X_train.corr().abs()

# select upper traingle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

<ipython-input-101-63e72ba12461>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [102]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.9999998)]
# to_drop_2 = [column for column in upper.columns if any(upper[column] > 0.999999)]
# to_drop_3 = [column for column in upper.columns if any(upper[column] > 0.9999995)]
# to_drop_4 = [column for column in upper.columns if any(upper[column] > 0.999998)]

In [103]:
# len(test_to_drop)
# len(to_drop_2)
# len(to_drop_3)
# len(to_drop_4)

In [104]:
# drop the columns
copy_sensor2_X_train.drop(to_drop, axis=1, inplace=True)

In [105]:
result = pd.concat([sensor1_X_train, copy_sensor2_X_train], axis=1)

In [106]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 1426 entries, A  to 2500 nm
dtypes: float64(1425), int64(1)
memory usage: 2.7 MB


In [107]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train_1, X_test, y_train_1, y_test = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

In [87]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train_1)

# transform training data
X_train_norm = norm.transform(X_train_1)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [88]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [108]:
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgm
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

clf_1 = MultiOutputRegressor(lgm.LGBMRegressor())
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=10, random_state=0))
# clf = MultiOutputRegressor(xgb.XGBRegressor(eval_metric='rmse'))
# clf = MultiOutputRegressor(xgb.XGBRegressor())
clf.fit(X_train_1, y_train_1)
clf_1.fit(X_train_stan_df, y_train_1)

MultiOutputRegressor(estimator=LGBMRegressor())

In [109]:
y_preds_1 = clf_1.predict(X_test_stan_df)
y_preds = clf.predict(X_test)

In [110]:
final_preds = (y_preds + y_preds_1)/2

In [111]:
mean_squared_error(y_test, final_preds)

28.83580669260924

In [ ]:
# 34.375625949329255 base and lightgbm
# 34.33242950236197 with to_drop best and lightgbm
# 33.3964077243079 WITH to_drop_2 and lightgbm
# 34.423633009416946 with to_drop_4 and lightgbm
# 58.402755542204375 with only 1 and no normalization and standardization and lightgbm
# 59.517577596796706 and normalization and lightgbm
# 54.69928458964822 with normalization of sensor 2 and lightgbm
# 57.84798370331507 with only sensor2 and lightgbm
# 34.33242950236197 both sensor 1 and 2 and drop 1 and lightgbm
# 33.155651438567624 with sensor 1 and 2 and drop 1 and normalization and lightgbm
# 33.500770656565486 with sensor 1 and 2 and drop 2 and normalization and lightgbm
# 31.53040079659907 with sensor 1 and 2 and drop 1 and normalization and standardization and lightgbm
# 41.57167433312246 with sensor 1 and 2 and drop 1 and normalization and standardization and xgboost
# 53.460146742703536 xgboost and only sensor 2 
# 39.34481110421532 entire dataset xgboost 
# 41.56526501057917 enitre dataset xgboost with normalization and standardization 
# 31.854466857032108 entire dataset randomforestregressor 
# 33.27334854065997 entire dataset randomforestregressor with normalization and standardization
# 32.14977971559776 with drop 1 randomforestregressor 
# 28.83580669260924 standardization, normalization, drop1 for lightgbm and entire dataset for randomforestregressor 

In [95]:
# for senssor 1
# corrmat = train.corr()
# g = sns.heatmap(train.corr())

# Testing sensor 1

In [7]:
find_corr_with_sensor1 =  pd.concat([sensor1_X_train, y_train], axis=1)

In [8]:
find_corr_with_sensor1.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Property_A,Property_B,Property_C,Property_D,Property_E,Property_F
0,101745.9975,292211.0199,38816.97313,5308.374022,161.262201,107.877266,384.928482,52565.24979,50.560870,50.918543,96.767202,9.663707,200.672050,71.013243,34,28.406635,5.99,0.115797,1.638,49.0,19.0,32.0
1,105712.6402,315908.0751,39888.72776,5431.548180,150.652845,117.086301,372.064549,50665.24752,53.176087,49.900172,93.678887,9.663707,199.593168,73.133041,35,29.386174,5.15,0.183483,1.482,47.0,16.0,37.0
2,102621.3309,304334.3675,42120.98104,5513.664286,161.262201,97.352655,422.530751,52748.36917,52.304348,50.918543,100.884955,9.663707,211.460870,74.192941,37,31.345253,5.55,0.173130,1.404,50.0,20.0,30.0
3,113446.2881,311356.2657,39195.84181,5428.306755,169.749685,99.983808,396.802883,53893.69764,52.304348,49.900172,97.796640,10.542226,209.303106,69.953344,35,30.365713,5.09,0.196011,1.638,50.0,15.0,35.0
4,109714.6033,322110.4501,38347.65382,5431.548180,152.774716,130.242065,346.336681,48784.11209,49.689130,44.808317,99.855517,8.785189,199.593168,73.133041,34,27.427096,4.73,0.203319,1.677,53.0,19.0,28.0


In [9]:
find_corr_with_sensor1.corr()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Property_A,Property_B,Property_C,Property_D,Property_E,Property_F
A,1.000000,0.264664,0.188617,0.405119,0.357634,0.512940,0.204806,0.543657,0.603615,0.338495,0.425290,0.360960,0.417989,-0.071394,-0.098181,0.398269,0.080794,0.070325,0.127471,0.002189,-0.089762,0.237878
B,0.264664,1.000000,0.344039,-0.036811,0.036292,-0.054857,-0.290961,-0.600555,-0.373769,-0.598723,-0.528746,-0.488214,-0.219089,0.313367,-0.171035,-0.422021,-0.180856,0.051470,-0.195266,-0.019165,0.124033,-0.279692
C,0.188617,0.344039,1.000000,0.044334,0.181336,-0.077408,0.010304,-0.034262,-0.016305,-0.116922,0.040448,0.168137,0.603684,0.416450,-0.183290,0.297668,-0.080401,0.303128,-0.321217,-0.401830,0.387641,0.171378
D,0.405119,-0.036811,0.044334,1.000000,0.245730,0.354628,0.018273,0.468872,0.436954,0.313447,0.334087,0.230937,0.342739,-0.280816,0.408448,0.353144,-0.175964,-0.282493,0.113386,0.158980,-0.250701,0.197624
E,0.357634,0.036292,0.181336,0.245730,1.000000,0.127125,0.143504,0.273652,0.344478,0.106621,0.233617,0.199823,0.399183,0.041047,0.088475,0.339590,-0.073887,-0.014070,-0.030076,-0.104074,0.059850,0.154546
F,0.512940,-0.054857,-0.077408,0.354628,0.127125,1.000000,0.103303,0.435231,0.442747,0.290313,0.316692,0.291318,0.228324,-0.242430,0.108957,0.293419,0.087170,-0.078315,0.116132,0.027130,-0.096014,0.178807
G,0.204806,-0.290961,0.010304,0.018273,0.143504,0.103303,1.000000,0.473508,0.435291,0.393534,0.405588,0.474942,0.176602,-0.210914,0.016552,0.267072,0.567346,-0.058775,0.031607,0.070023,-0.136170,0.157399
H,0.543657,-0.600555,-0.034262,0.468872,0.273652,0.435231,0.473508,1.000000,0.822620,0.796868,0.813874,0.771909,0.561501,-0.317377,0.162924,0.710268,0.221244,-0.032511,0.210457,0.008566,-0.180291,0.465334
I,0.603615,-0.373769,-0.016305,0.436954,0.344478,0.442747,0.435291,0.822620,1.000000,0.605575,0.697143,0.626054,0.518600,-0.207462,0.119222,0.602065,0.209835,-0.019703,0.176898,-0.001249,-0.153119,0.421068
J,0.338495,-0.598723,-0.116922,0.313447,0.106621,0.290313,0.393534,0.796868,0.605575,1.000000,0.708081,0.614609,0.351635,-0.245827,0.175330,0.548863,0.249145,-0.045774,0.181380,0.010537,-0.151616,0.381234


# for sensor 1

In [10]:
# for property A = B,D,G,H,I,J,K,L,N
# for property b = C,D,L,M,N,O
# for property c = A,B,C,D,F,H,I,J,K,L,M,N
# for property d = c,d,e,m,n,o,p
# for property e = b,c,d,g,h,i,j,m,n,o,p
# for property f = a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p

# For sensor 2

In [19]:
sensor2_X_train.head()

350 nm    351 nm    352 nm    353 nm    354 nm    355 nm    356 nm  \
0  0.254557  0.251017  0.248123  0.247993  0.245087  0.240057  0.240407   
1  0.258670  0.256057  0.253953  0.252913  0.251137  0.248653  0.247337   
2  0.257663  0.257633  0.256357  0.251743  0.249327  0.249160  0.246877   
3  0.252360  0.251830  0.250630  0.244980  0.242393  0.243070  0.242150   
4  0.261113  0.259197  0.256343  0.250743  0.248863  0.250517  0.249780   

     357 nm    358 nm    359 nm    360 nm    361 nm    362 nm    363 nm  \
0  0.240223  0.237893  0.234223  0.230290  0.227303  0.223847  0.220810   
1  0.244513  0.239613  0.237633  0.236100  0.232827  0.229323  0.225900   
2  0.244930  0.243797  0.240653  0.237347  0.235663  0.233120  0.230050   
3  0.240353  0.237693  0.234507  0.231327  0.228807  0.225880  0.222690   
4  0.247317  0.243230  0.241020  0.238663  0.234150  0.231620  0.229417   

     364 nm    365 nm    366 nm    367 nm    368 nm    369 nm    370 nm  \
0  0.220270  0.217730  0.213287  0.210867  0.208923  0.207137  0.205603   
1  0.223163  0.219783  0.215857  0.212527  0.210903  0.211043  0.208287   
2  0.227090  0.223297  0.219210  0.218170  0.216190  0.212427  0.211397   
3  0.219407  0.216427  0.213767  0.211097  0.208440  0.205893  0.204687   
4  0.224253  0.220663  0.218927  0.216997  0.214277  0.210780  0.210187   

     371 nm    372 nm    373 nm    374 nm    375 nm    376 nm    377 nm  \
0  0.204247  0.203067  0.199647  0.195987  0.194737  0.194253  0.193760   
1  0.205357  0.203587  0.200517  0.197317  0.195513  0.194250  0.193140   
2  0.210423  0.208210  0.205010  0.201833  0.199953  0.199657  0.199767   
3  0.203047  0.199943  0.198157  0.196773  0.194543  0.192890  0.191543   
4  0.209357  0.206647  0.202680  0.199140  0.198313  0.198217  0.198020   

     378 nm    379 nm    380 nm    381 nm    382 nm    383 nm    384 nm  \
0  0.190877  0.188343  0.186567  0.184340  0.182647  0.181780  0.180480   
1  0.191033  0.189147  0.187523  0.184737  0.182560  0.181603  0.179967   
2  0.196233  0.193187  0.191357  0.190170  0.188570  0.186200  0.185500   
3  0.189390  0.187193  0.185053  0.182520  0.180817  0.180293  0.178423   
4  0.195410  0.192060  0.188663  0.187297  0.185833  0.183530  0.182227   

     385 nm    386 nm    387 nm    388 nm    389 nm    390 nm    391 nm  \
0  0.179083  0.177760  0.177110  0.176773  0.176230  0.176013  0.175913   
1  0.178393  0.177343  0.176803  0.176500  0.176070  0.175357  0.174560   
2  0.184887  0.183237  0.181863  0.180833  0.180223  0.180227  0.180470   
3  0.176400  0.175013  0.173620  0.172597  0.172460  0.172237  0.171833   
4  0.181277  0.180263  0.179430  0.178763  0.178203  0.177857  0.177560   

     392 nm    393 nm    394 nm    395 nm    396 nm    397 nm    398 nm  \
0  0.175293  0.175133  0.175370  0.174983  0.174797  0.174983  0.174603   
1  0.174230  0.173847  0.173403  0.173153  0.173313  0.173853  0.173503   
2  0.180073  0.179200  0.178180  0.178503  0.178813  0.178713  0.178247   
3  0.171143  0.170630  0.170377  0.170500  0.170310  0.169667  0.169717   
4  0.176853  0.176403  0.176307  0.176443  0.176450  0.176220  0.176017   

     399 nm    400 nm    401 nm    402 nm    403 nm    404 nm    405 nm  \
0  0.174213  0.174093  0.174073  0.174180  0.174443  0.174183  0.173697   
1  0.173003  0.172720  0.172800  0.172837  0.172533  0.172353  0.172067   
2  0.177907  0.178007  0.178163  0.178013  0.177270  0.177063  0.177130   
3  0.169620  0.168963  0.168967  0.168970  0.168490  0.168117  0.167847   
4  0.175727  0.175290  0.175173  0.175053  0.174697  0.174707  0.174553   

     406 nm    407 nm    408 nm    409 nm    410 nm    411 nm    412 nm  \
0  0.173400  0.173037  0.172717  0.172690  0.172910  0.173233  0.173230   
1  0.171347  0.170917  0.170810  0.171010  0.171347  0.171670  0.171467   
2  0.177190  0.176607  0.175783  0.175840  0.175913  0.175973  0.176153   
3  0.167677  0.167250  0.166700  0.166350  0.166260  0.166390  0.166627   
4  0.173463  0.173253 

In [21]:
list_1 = sensor2_X_train.columns

In [23]:
list_2 = []

for i in list_1:
    if int(i[:4]) % 50 == 0:
        list_2.append(i)

In [455]:
list_3 = []

for i in list_1:
    if int(i[:4]) in range(550,600):
        list_3.append(i)

In [25]:
sensor2_X_train_with_mod_50 = sensor2_X_train[list_2]

In [159]:
sensor2_X_train_550_around = sensor2_X_train[list_3]

In [168]:
find_corr_with_sensor2_property_c = pd.concat([sensor2_X_train_550_around,y_train])

In [28]:
find_corr_with_sensor2 =  pd.concat([sensor2_X_train_with_mod_50, y_train], axis=1)

In [30]:
find_corr_with_sensor2.corr()

,350 nm,400 nm,450 nm,500 nm,550 nm,600 nm,650 nm,700 nm,750 nm,800 nm,850 nm,900 nm,950 nm,1000 nm,1050 nm,1100 nm,1150 nm,1200 nm,1250 nm,1300 nm,1350 nm,1400 nm,1450 nm,1500 nm,1550 nm,1600 nm,1650 nm,1700 nm,1750 nm,1800 nm,1850 nm,1900 nm,1950 nm,2000 nm,2050 nm,2100 nm,2150 nm,2200 nm,2250 nm,2300 nm,2350 nm,2400 nm,2450 nm,2500 nm,Property_A,Property_B,Property_C,Property_D,Property_E,Property_F
350 nm,1.000000,0.961629,0.952065,0.949359,0.931939,0.917136,0.906927,0.897604,0.887391,0.878801,0.875320,0.873200,0.868357,0.863372,0.858474,0.853550,0.848597,0.844023,0.839657,0.835381,0.831894,0.832783,0.828297,0.823410,0.818429,0.814669,0.811178,0.808532,0.806384,0.804045,0.800894,0.806024,0.801774,0.794308,0.788740,0.785679,0.785456,0.789286,0.785603,0.787004,0.788922,0.787332,0.788485,0.785619,-0.232976,-0.272670,0.091289,0.349950,-0.261309,-0.357106
400 nm,0.961629,1.000000,0.984056,0.957105,0.897382,0.848104,0.822782,0.803839,0.785174,0.770843,0.765006,0.761055,0.754061,0.748760,0.741721,0.734058,0.726698,0.720350,0.714872,0.710138,0.706998,0.713473,0.707560,0.700855,0.694619,0.690479,0.687059,0.684822,0.683400,0.681749,0.679427,0.702922,0.702719,0.690000,0.681631,0.677516,0.679133,0.688388,0.681763,0.685727,0.690902,0.693456,0.699232,0.704048,-0.160766,-0.183758,0.046353,0.267690,-0.177888,-0.333168
450 nm,0.952065,0.984056,1.000000,0.990401,0.948976,0.901598,0.874902,0.855294,0.836614,0.822492,0.815871,0.810505,0.803066,0.799399,0.792673,0.784713,0.777107,0.770547,0.764919,0.760133,0.757062,0.764575,0.758221,0.751000,0.744201,0.739761,0.736187,0.733909,0.732513,0.730870,0.728818,0.758607,0.760990,0.747440,0.738759,0.734567,0.736478,0.746580,0.739471,0.744019,0.749795,0.753363,0.759830,0.765686,-0.189770,-0.150417,-0.039448,0.176982,-0.088283,-0.300177
500 nm,0.949359,0.957105,0.990401,1.000000,0.981083,0.947320,0.926478,0.910405,0.894960,0.883284,0.877501,0.872368,0.865807,0.863235,0.857435,0.850275,0.843431,0.837491,0.832367,0.827972,0.825119,0.832041,0.825891,0.818991,0.812442,0.808103,0.804608,0.802299,0.800836,0.799175,0.797282,0.825607,0.827534,0.815049,0.806960,0.803101,0.804840,0.814203,0.807587,0.811693,0.817100,0.820331,0.826083,0.830641,-0.248334,-0.160579,-0.076240,0.144350,-0.052718,-0.297365
550 nm,0.931939,0.897382,0.948976,0.981083,1.000000,0.989423,0.976617,0.965572,0.954760,0.946033,0.940303,0.934714,0.928871,0.926925,0.922330,0.916677,0.911160,0.906114,0.901456,0.897174,0.894032,0.898184,0.892650,0.886432,0.880325,0.876019,0.872452,0.869883,0.868025,0.866023,0.863953,0.884594,0.884400,0.874165,0.867443,0.864185,0.865142,0.871905,0.867040,0.870202,0.874109,0.876072,0.880080,0.882024,-0.315603,-0.182378,-0.108031,0.135676,-0.044316,-0.293727
600 nm,0.917136,0.848104,0.901598,0.947320,0.989423,1.000000,0.996991,0.991864,0.985805,0.980040,0.975585,0.971161,0.966624,0.964658,0.961051,0.956827,0.952494,0.948251,0.944024,0.939865,0.936443,0.937727,0.932787,0.927435,0.922032,0.917903,0.914360,0.911565,0.909318,0.907024,0.904736,0.915497,0.912163,0.904558,0.899216,0.896528,0.896618,0.900895,0.897527,0.899417,0.901894,0.902029,0.904033,0.902948,-0.370070,-0.210414,-0.092966,0.162521,-0.071551,-0.301496
650 nm,0.906927,0.822782,0.874902,0.926478,0.976617,0.996991,1.000000,0.998652,0.995424,0.991474,0.988272,0.985001,0.981383,0.979593,0.976628,0.973174,0.969457,0.965644,0.961697,0.957694,0.954246,0.954287,0.949596,0.944759,0.939798,0.935853,0.932405,0.929565,0.927196,0.924848,0.922540,0.928878,0.924276,0.918222,0.913660,0.911273,0.911014,0.914256,0.911419,0.912566,0.914440,0.913585,0.914503,0.911638,-0.397521,-0.220753,-0.078038,0.169878,-0.079412,-0.302535
700 nm,0.897604,0.803839,0.855294,0.910405,0.965572,0.991864,0.998652,1.000000,0.998961,0.996561,0.994366,0.991935,0.989059,0.987509,0.985087,0.982286,0.979136,0.975762,0.972155,0.968402,0.965050,0.964287,0.959913,0.955557,0.951015,0.947294,0.943985,0.941186,0.938795,0.936456,0.934170,0.937264,0.931837,0.927060,0.923160,0.921027,0.920501,0.922898,0.920530,0.92

In [ ]:
# For property_A entire shit 
# For property_B entire shit
# for property_c look around 550
# for property_d entire shit 
# for property_e around 350-450,800-1850
# for property_f entire shit 

# Predicting Individual classes 

## Predicting a first 

In [391]:
# sensor 1 for property A = B,D,G,H,I,J,K,L,N
# sensor 2 For property_A entire shit 

In [392]:
x_train_sensor1 = sensor1_X_train[['B','D','G','H','I','J','K','L','N']]

In [393]:
y_train_a = y_train['Property_A']

In [394]:
x_train_sensor2 = sensor2_X_train

### Predicting seperately then taking the average

In [395]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(x_train_sensor1, y_train_a, test_size=0.30, random_state=42)

In [396]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(x_train_sensor2, y_train_a, test_size=0.30, random_state=42)

In [397]:
import lightgbm as lgm

clf_1 = lgm.LGBMRegressor()
clf_1.fit(X_train_1, y_train_1)

LGBMRegressor()

In [398]:
y_preds_1 = clf_1.predict(X_test_1)

In [399]:
clf_2 = lgm.LGBMRegressor()
clf_2.fit(X_train_2, y_train_2)

LGBMRegressor()

In [400]:
y_preds_2 = clf_2.predict(X_test_2)

In [401]:
mean_squared_error(y_test_2, y_preds_2)

0.3040020816099806

In [402]:
mean_squared_error(y_test_1, y_preds_1)

0.2292887963195052

In [403]:
y_final = (y_preds_2 + y_preds_1)/2

In [404]:
mean_squared_error(y_test_1, y_final)

0.22400622375426782

### Predicting together

In [405]:
x_train_a_together = pd.concat([x_train_sensor1, x_train_sensor2], axis=1)

In [406]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_a_together, y_train_a, test_size=0.30, random_state=42)

In [407]:
clf_3 = lgm.LGBMRegressor()
clf_3.fit(X_train_3, y_train_3)

LGBMRegressor()

In [408]:
y_preds_3 = clf_3.predict(X_test_3)

In [409]:
mean_squared_error(y_test_3, y_preds_3)

0.19013825891561864

### predicting together with normalization and standardization

In [410]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_a_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [411]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [432]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

# clf_1 = (lgm.LGBMRegressor(num_leaves=20 , max_depth=4))
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor())
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.08)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
# clf_2.fit(X_train_stan_df, y_train_3)
# clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
# clf_7.fit(X_train_stan_df, y_train_3)
# clf_8.fit(X_train_stan_df, y_train_3)

Ridge(alpha=0.08)

In [433]:
y_preds_3 = clf_5.predict(X_test_stan_df)
mean_squared_error(y_test_3, y_preds_3)

0.12083786315253413

In [68]:
# lightgbm 0.18198466990829576
# random forest 0.1715168560359609
# xgb 0.1951914905244933
# lasso 0.37394187755102026
# ridge 0.13418688713317348
# linear regression 0.16516285239105377
# svr 0.28239058871930456
# elasticnet 0.37394187755102026

0.18198466990829576

## Predicting b 

In [339]:
# sensor 1 for property b = C,D,L,M,N,O
# sensor 2 For property_b entire shit 

In [340]:
x_train_b_sensor1 = sensor1_X_train[['C','D','L','M','N','O']]

In [341]:
y_train_b = y_train['Property_B']

In [342]:
x_train_b_sensor2 = sensor2_X_train

### Predicting seperately then taking the average

In [343]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(x_train_b_sensor1, y_train_b, test_size=0.30, random_state=42)

In [344]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(x_train_b_sensor2, y_train_b, test_size=0.30, random_state=42)

In [345]:
import lightgbm as lgm

clf_1 = lgm.LGBMRegressor()
clf_1.fit(X_train_1, y_train_1)

LGBMRegressor()

In [346]:
y_preds_1 = clf_1.predict(X_test_1)

In [347]:
clf_2 = lgm.LGBMRegressor()
clf_2.fit(X_train_2, y_train_2)

LGBMRegressor()

In [348]:
y_preds_2 = clf_2.predict(X_test_2)

In [349]:
mean_squared_error(y_test_2, y_preds_2)

0.7463645013850703

In [350]:
mean_squared_error(y_test_1, y_preds_1)

0.5429867662377011

In [351]:
y_final = (y_preds_2 + y_preds_1)/2

In [352]:
mean_squared_error(y_test_1, y_final)

0.5340477369373791

### Predicting together

In [353]:
x_train_b_together = pd.concat([x_train_b_sensor1, x_train_b_sensor2], axis=1)

In [354]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_b_together, y_train_b, test_size=0.30, random_state=42)

In [355]:
clf_3 = lgm.LGBMRegressor()
clf_3.fit(X_train_3, y_train_3)

LGBMRegressor()

In [356]:
y_preds_3 = clf_3.predict(X_test_3)

In [357]:
mean_squared_error(y_test_3, y_preds_3)

0.5302857591706125

### predicting together with normalization and standardization

In [358]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_b_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [359]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [388]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

# clf_1 = (lgm.LGBMRegressor(num_leaves=20 , max_depth=4))
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor())
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.022)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
# clf_2.fit(X_train_stan_df, y_train_3)
# clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
# clf_7.fit(X_train_stan_df, y_train_3)
# clf_8.fit(X_train_stan_df, y_train_3)

Ridge(alpha=0.022)

In [389]:
y_preds_3 = clf_5.predict(X_test_stan_df)
mean_squared_error(y_test_3, y_preds_3)

0.1276738394451057

In [362]:
# lightgbm 0.48098115752695625
#  random forest 0.6447582097094091
#  xgb 0.41259081610663645
# lasso 0.6526428697105817
# ridge 0.251869370516629
#  linear regression 0.20335741660030346
# 0.6029370948638095 svm
# 0.6526428697105817 elastic fit
# clf_5 = Ridge(alpha=0.022) 0.1276738394451057


## Predicting c

In [456]:
# for property c = A,B,C,D,F,H,I,J,K,L,M,N
# sensor 2 # for property_c look around 550

In [457]:
x_train_c_sensor1 = sensor1_X_train[['B','C','D','F','H','I','J','K','L','M','N']]

In [458]:
y_train_c = y_train['Property_C']

In [459]:
x_train_c_sensor2 = sensor2_X_train[list_3]

In [460]:
x_train_c_together = pd.concat([x_train_c_sensor1, x_train_c_sensor2], axis=1)

### predicting together with normalization and standardization

In [461]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_c_together, y_train_c, test_size=0.30, random_state=42)

In [462]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_c_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [463]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [486]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

# clf_1 = (lgm.LGBMRegressor(num_leaves=20 , max_depth=4))
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor(eta=0.5,max_depth=6,eval_metric = 'rmse'))
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.063)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
# clf_2.fit(X_train_stan_df, y_train_3)
clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
# clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
# clf_7.fit(X_train_stan_df, y_train_3)
# clf_8.fit(X_train_stan_df, y_train_3)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.5,
             eval_metric='rmse', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0, ...)

In [487]:
y_preds_3 = clf_3.predict(X_test_stan_df)
mean_squared_error(y_test_3, y_preds_3)

0.05990128216756188

In [191]:
# light gbm 0.07744226515810106
# random forest regressor 0.07111063468488386
# xgb 0.06100447694933065
# lasso 0.10148668125170068
# ridge 0.08160352973363215
# linear 0.08819277112631875

0.07744226515810106

## Predicting d

In [291]:
# sensor 1 for property d = c,d,e,m,n,o,p
# sensor 2 For property_d entire shit 

In [292]:
x_train_d_sensor1 = sensor1_X_train[['C','D','E','M','N','O','P']]

In [293]:
y_train_d = y_train['Property_D']

In [294]:
x_train_d_sensor2 = sensor2_X_train

In [295]:
x_train_d_together = pd.concat([x_train_d_sensor1, x_train_d_sensor2], axis=1)

### predicting together with normalization and standardization

In [296]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_d_together, y_train_d, test_size=0.30, random_state=42)

In [297]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_d_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [298]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [337]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

# clf_1 = (lgm.LGBMRegressor(num_leaves=20 , max_depth=4))
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor())
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.063)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
# clf_2.fit(X_train_stan_df, y_train_3)
# clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
clf_7.fit(X_train_stan_df, y_train_3)
clf_8.fit(X_train_stan_df, y_train_3)

ElasticNet()

In [338]:
y_preds_3 = clf_5.predict(X_test_stan_df)
mean_squared_error(y_test_3, y_preds_3)

60.644571120198115

In [154]:
# lightgbm 84.72842855328177
# random forest 86.98684642164167
# xgb 85.50891357968845
# lasso 145.41777543846106
# ridge 68.06161763289921
# 60.644571120198115 ridge with 0.063
# linear regression 70.34758857745368
# svr 300.96554907604286
#  elasticnet 137.9091919214232

84.72842855328177

In [259]:
# get a list of base models
def get_models():
    models = list()
    models.append(('linear regression' , LinearRegression()))
    models.append(('ridge', Ridge(alpha=0.0013)))
#     models.append(('lgm', lgm.LGBMRegressor()))
    return models


In [260]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
 # fit and evaluate the models
    scores = list()
    for name, model in models:
 # fit the model
        model.fit(X_train, y_train)
 # evaluate the model
        yhat = model.predict(X_val)
        mse = mean_squared_error(y_val, yhat)
 # store the performance
        scores.append(mse)
 # report model performance
    return scores

In [261]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_3)
# make predictions on test set
yhat = ensemble.predict(X_test_stan_df)
# evaluate predictions
score = mean_squared_error(y_test_3, yhat)
print('Weighted Avg MAE: %.3f' % (score))
# evaluate each standalone model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
for i in range(len(models)):
 print('>%s: %.3f' % (models[i][0], scores[i]))
# evaluate equal weighting
ensemble = VotingRegressor(estimators=models)
ensemble.fit(X_train_stan_df, y_train_3)
yhat = ensemble.predict(X_test_stan_df)
score = mean_squared_error(y_test_3, yhat)
print('Voting MAE: %.3f' % (score))

[86.13547213177594, 71.28668599790599]
Weighted Avg MAE: 74.742
>linear regression: 86.135
>ridge: 71.287
Voting MAE: 74.000


## Predicting e

In [264]:
# sensor 1 for property e = b,c,d,g,h,i,j,m,n,o,p
# sensor 2 for property_e around 350-450,800-1850

In [265]:
x_train_e_sensor1 = sensor1_X_train[['B','C','D','G','H','I','J','M','N','O','P']]

In [266]:
y_train_e = y_train['Property_E']

In [267]:
list_1 = sensor2_X_train.columns

In [268]:
list_4 = []

for i in list_1:
    if int(i[:4]) in range(350,450):
        list_4.append(i)

for i in list_1:
    if int(i[:4]) in range(800,1850):
        list_4.append(i)

In [269]:
x_train_e_sensor2 = sensor2_X_train[list_4]

In [270]:
x_train_e_together = pd.concat([x_train_e_sensor1, x_train_e_sensor2], axis=1)

### predicting together with normalization and standardization

In [271]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_e_together, y_train_e, test_size=0.30, random_state=42)

In [272]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_e_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [273]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [289]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

# clf_1 = (lgm.LGBMRegressor(num_leaves=20 , max_depth=4))
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor())
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.0013)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
# clf_2.fit(X_train_stan_df, y_train_3)
# clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
# clf_7.fit(X_train_stan_df, y_train_3)
# clf_8.fit(X_train_stan_df, y_train_3)

Ridge(alpha=0.0013)

In [290]:
y_preds_3 = clf_5.predict(X_test_stan_df)
mean_squared_error(y_test_3, y_preds_3)

71.28668599790599

In [278]:
# 86.10672476704988 light gbm
# 94.7427452951249 random forest
# 144.23025038189778 xgb
# 131.85470955300605 lasso
# 91.9855964457914 ridge
# 71.28668599790599 ridge with alpha 0.0013
# 86.13547213177594 linear
# 250 svr 
#  126 elasticnet

In [259]:
# get a list of base models
def get_models():
    models = list()
    models.append(('linear regression' , LinearRegression()))
    models.append(('ridge', Ridge(alpha=0.0013)))
#     models.append(('lgm', lgm.LGBMRegressor()))
    return models


In [260]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
 # fit and evaluate the models
    scores = list()
    for name, model in models:
 # fit the model
        model.fit(X_train, y_train)
 # evaluate the model
        yhat = model.predict(X_val)
        mse = mean_squared_error(y_val, yhat)
 # store the performance
        scores.append(mse)
 # report model performance
    return scores

In [261]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_3)
# make predictions on test set
yhat = ensemble.predict(X_test_stan_df)
# evaluate predictions
score = mean_squared_error(y_test_3, yhat)
print('Weighted Avg MAE: %.3f' % (score))
# evaluate each standalone model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
for i in range(len(models)):
 print('>%s: %.3f' % (models[i][0], scores[i]))
# evaluate equal weighting
ensemble = VotingRegressor(estimators=models)
ensemble.fit(X_train_stan_df, y_train_3)
yhat = ensemble.predict(X_test_stan_df)
score = mean_squared_error(y_test_3, yhat)
print('Voting MAE: %.3f' % (score))

[86.13547213177594, 71.28668599790599]
Weighted Avg MAE: 74.742
>linear regression: 86.135
>ridge: 71.287
Voting MAE: 74.000


86.10672476704988

## Predicting f

In [31]:
# sensor 1 for property f = a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
# sensor 2 For property_f entire shit 

In [32]:
x_train_f_sensor1 = sensor1_X_train

In [33]:
y_train_f = y_train['Property_F']

In [34]:
x_train_f_sensor2 = sensor2_X_train

In [35]:
x_train_f_together = pd.concat([x_train_f_sensor1, x_train_f_sensor2], axis=1)

### predicting together with normalization and standardization

In [36]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(x_train_f_together, y_train_f, test_size=0.30, random_state=42)

In [37]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train_f_together)

# transform training data
X_train_norm = norm.transform(X_train_3)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test_3)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [38]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [151]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet

clf_1 = (lgm.LGBMRegressor())
clf_2 = (RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0))
clf_3 = (xgb.XGBRegressor())
clf_4 = Lasso()
clf_5 = Ridge(alpha=0.915)
clf_6 = LinearRegression()
clf_7 = SVR()
clf_8 = ElasticNet()
# clf_1.fit(X_train_stan_df, y_train_3)
clf_2.fit(X_train_stan_df, y_train_3)
# clf_3.fit(X_train_stan_df, y_train_3)
# clf_4.fit(X_train_stan_df, y_train_3)
# clf_5.fit(X_train_stan_df, y_train_3)
# clf_6.fit(X_train_stan_df, y_train_3)
# clf_7.fit(X_train_stan_df, y_train_3)
# clf_8.fit(X_train_stan_df, y_train_3)

RandomForestRegressor(max_depth=10, n_estimators=38, random_state=0)

In [152]:
y_preds_3 = clf_2.predict(X_test_stan_df)
# y_preds_4 = clf_1.predict(X_test_stan_df)

In [164]:
# get a list of base models
def get_models():
    models = list()
    models.append(('random forest' , RandomForestRegressor(n_estimators=38,max_depth=10, random_state=0)))
    models.append(('ridge', Ridge(alpha=0.915)))
#     models.append(('lgm', lgm.LGBMRegressor()))
    return models


In [165]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
 # fit and evaluate the models
    scores = list()
    for name, model in models:
 # fit the model
        model.fit(X_train, y_train)
 # evaluate the model
        yhat = model.predict(X_val)
        mse = mean_squared_error(y_val, yhat)
 # store the performance
        scores.append(mse)
 # report model performance
    return scores

In [166]:
from sklearn.ensemble import VotingRegressor

models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
print(scores)
# create the ensemble
ensemble = VotingRegressor(estimators=models, weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(X_train_stan_df, y_train_3)
# make predictions on test set
yhat = ensemble.predict(X_test_stan_df)
# evaluate predictions
score = mean_squared_error(y_test_3, yhat)
print('Weighted Avg MAE: %.3f' % (score))
# evaluate each standalone model
scores = evaluate_models(models, X_train_stan_df, X_test_stan_df, y_train_3, y_test_3)
for i in range(len(models)):
 print('>%s: %.3f' % (models[i][0], scores[i]))
# evaluate equal weighting
ensemble = VotingRegressor(estimators=models)
ensemble.fit(X_train_stan_df, y_train_3)
yhat = ensemble.predict(X_test_stan_df)
score = mean_squared_error(y_test_3, yhat)
print('Voting MAE: %.3f' % (score))

[19.114413541898337, 19.803592206943]
Weighted Avg MAE: 18.035
>random forest: 19.114
>ridge: 19.804
Voting MAE: 18.029


In [134]:
# final_preds = (y_preds_3+y_preds_4)/2

In [153]:
mean_squared_error(y_test_3, y_preds_3)
# 23.062043664764623 lightbgm
# 21.178296815117477 randomforest
# 19.114413541898337 with 38 random forest estimators
# 31 for xgboost
# 21.35384330913589 for both random and lgm
# 19.80495781959146 ridge 
# 26 for lasso 
# 34.71444135146908 linear regression 
# 27.16547205674925 svm
# 25.3466624886579 elasticnet

19.114413541898337

In [220]:
(23.062043664764623 + 86.10672476704988 + 84.72842855328177 + 0.07744226515810106 + 0.3124515993965998 + 0.18198466990829576) / 6

32.41151258659322

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

clf_1 = MultiOutputRegressor(lgm.LGBMRegressor())
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=10, random_state=0))
clf = MultiOutputRegressor(xgb.XGBRegressor())

In [389]:
(0.13418688713317348 + 19.80495781959146 + 86.10672476704988 + 68.06161763289921 + 0.06100447694933065 + 0.20335741660030346)/6

29.06197483337056

In [488]:
(0.12083786315253413 + 0.1276738394451057 + 0.05990128216756188 + 60.644571120198115 + 18.029 + 71.28668599790599) / 6

25.044778350478214